In [2]:
%%capture
!pip install pyxnat
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"
!pip install sevenbridges-python

In [6]:
import os

In [31]:
from pyxnat import Interface
from glob import glob
import shutil


def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM')
      if not xres.exists():
        xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary')
      xres.get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist


In [34]:
xhost = 'https://pixnat.mskcc.org'
user = 'locastre'

#example of dictionary to describe the data you want to pull locally from XNAT,
# one entry per experiment, iterate over scans
scandict = [{'proj':'RTOG0617','subj':'SPI_XNAT_S00220','exp':'SPI_XNAT_E00372', 'scan_list':['1','2']}]

dataPath = '/content/images'
os.makedirs(dataPath,exist_ok=True)

folderList = getXNATData(xhost,user,scandict,dataPath)

··········
['/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-60-63l6bd.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-116-sw4osw.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-143-1me363m.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-31-vxavj9.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-51-1sybofw.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-65-199bvog.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-38-1366mfy.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-91-gsz3fe.dcm', '/content/images/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855-1-59-1orvtym.dcm', '/content/images/DICOM/1.3.

In [13]:
from cerr import plan_container as pc

In [35]:
# Import DICOM to planC
planC = pc.loadDcmDir(folderList[0])

('0617-429991', '0617-429991', '1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855', '1.3.6.1.4.1.14519.5.2.1.6329.6468.159702675618311983480465966977', 'RTSTRUCT', '', '', '', '', '', '', '')
('0617-429991', '0617-429991', '1.3.6.1.4.1.14519.5.2.1.6329.6468.672376632365624629222827737855', '1.3.6.1.4.1.14519.5.2.1.6329.6468.299190288252230715482284274291', 'CT', '', '', '', '', '', '', '')


In [41]:
from cerr.viewer import showMplNb
showMplNb(planC=planC, scan_nums=0, struct_nums = [0,1,2,3,4,5], windowCenter=-400, windowWidth=2000)

Output()